In [25]:
import numpy as np
import subprocess
import os
import cv2
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract

# From imutils
def resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and grab the image size
    dim = None
    (h, w) = image.shape[:2]
    # if both the width and height are None, then return the original image
    if width is None and height is None:
        return image
    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the dimensions
        r = height / float(h)
        dim = (int(w * r), height)
    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the dimensions
        r = width / float(w)
        dim = (width, int(h * r))
    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)
    # return the resized image
    return resized

#cv2.imshow("quest_tracker", img_rgb)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [26]:
img_rgb = cv2.imread('test9.jpg')
img = cv2.cvtColor(np.asarray(img_rgb), cv2.COLOR_BGR2GRAY)

template_rgb = cv2.imread('msq_icon.png', cv2.IMREAD_UNCHANGED)
template = cv2.Canny(template_rgb, 50, 200)
(tH, tW) = template.shape[:2]


In [38]:
img_hsv = cv2.cvtColor(np.asarray(img_rgb), cv2.COLOR_BGR2HSV)
template_hsv = cv2.cvtColor(np.asarray(template_rgb), cv2.COLOR_BGR2HSV)
    
lower_red = np.array([15,20,100])
upper_red = np.array([50,255,255])

mask = cv2.inRange(img_hsv, lower_red, upper_red)
template_mask = cv2.inRange(template_hsv, lower_red, upper_red)

res = cv2.bitwise_and(img_hsv, img_hsv, mask= mask)
template_res = cv2.bitwise_and(template_hsv, template_hsv, mask= template_mask)

cv2.imshow("Image", res)
cv2.waitKey(0)

-1